<a href="https://colab.research.google.com/github/ajayverma23/GenAI/blob/main/LLaMA_13B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip3 install llama-cpp-python
!pip3 install huggingface-hub
!pip3 install sentence-transformers langchain langchain-experimental
!huggingface-cli download TheBloke/Llama-2-7b-Chat-GGUF llama-2-7b-chat.Q4_K_M.gguf --local-dir /content --local-dir-use-symlinks False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 22.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.28-cp310-cp310-manylinux_2_35_x86_64.whl size=8739304 sha256=01bccb72ed02d1241c44d26f478294470cb21aa1a2b7878c91ce92d090ba0886
  Stored in directory: /root/.cache/pip/wheels/93/6e/a9/478cce089dc2a082bdcffe468a1c65465c91b25d911b30da82
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from langchain.llms import LlamaCpp

dir(LlamaCpp)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [4]:
#load the model and test that it works:

from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

n_gpu_layers = 40
n_batch = 512

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path="/content/llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.1,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [5]:
llm("What is the distance to the Moon? Write the short answer.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




Answer: The average distance from Earth to the Moon is about 384,400 kilometers (238,900 miles).

'\n\nAnswer: The average distance from Earth to the Moon is about 384,400 kilometers (238,900 miles).'

In [6]:
#1. LLM Chain
#LCEL (LangChain Expression Language) create a prompt and bind it to a language model
# added an StrOutputParser instance to clean the output text
#callbacks is an optional parameter that allows us to debug the chain

from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler

template = """<s>[INST] <<SYS>>
Provide a correct and short answer to the question.
<</SYS>>
{question} [/INST]"""

prompt = PromptTemplate(template=template, input_variables=["question"])

chain = prompt | llm | StrOutputParser()

chain.invoke({"question": "What is the distance to the Moon?"},
             config={
    # "callbacks": [ConsoleCallbackHandler()]
})

Llama.generate: prefix-match hit


  The average distance from Earth to the Moon is about 384,400 kilometers (238,900 miles).

'  The average distance from Earth to the Moon is about 384,400 kilometers (238,900 miles).'

In [7]:
#2 Combining the chains - added a second chain that uses the output of the first one as an input.

template2 = """<s>[INST] <<SYS>>
Use the summary {summary} and give 2 one sentence examples of
practical applications of the subject [/INST]
<</SYS>>
[/INST]
"""

prompt2 = PromptTemplate(template=template2, input_variables=["summary"],)

chain2 = {"summary": prompt | llm | StrOutputParser()} \
         | prompt2 | llm | StrOutputParser()

chain2.invoke({"question": "What is the distance to the Moon?"},
              config={
    # "callbacks": [ConsoleCallbackHandler()]
})

  The average

Llama.generate: prefix-match hit


 distance from Earth to the Moon is about 384,400 kilometers (238,900 miles).

Llama.generate: prefix-match hit


The average distance from Earth to the Moon is approximately 384,400 kilometers (238,900 miles), which has numerous practical applications in various fields. Here are two examples:
1. Space Exploration: The distance between Earth and the Moon is a crucial factor in determining the trajectory of spacecraft during lunar missions. By accurately measuring this distance, scientists can plan and execute successful landing and return missions to the Moon.
2. Satellite Communication: The distance between Earth and the Moon affects the signal strength and quality of satellite communications. By understanding this distance, engineers can design and operate communication systems that provide reliable and high-quality connections between Earth and space-based satellites.

'The average distance from Earth to the Moon is approximately 384,400 kilometers (238,900 miles), which has numerous practical applications in various fields. Here are two examples:\n1. Space Exploration: The distance between Earth and the Moon is a crucial factor in determining the trajectory of spacecraft during lunar missions. By accurately measuring this distance, scientists can plan and execute successful landing and return missions to the Moon.\n2. Satellite Communication: The distance between Earth and the Moon affects the signal strength and quality of satellite communications. By understanding this distance, engineers can design and operate communication systems that provide reliable and high-quality connections between Earth and space-based satellites.'

In [ ]:
#enable ConsoleCallbackHandler -

'''[llm/start] Exiting Prompt run with output:
"<s>[INST] <<SYS>>\nProvide a correct and short answer to the question.\n<</SYS>>\nWhat is the distance to the Moon? [/INST]"

Exiting LLM run with output:
"The average distance from Earth to the Moon is about 384,400 kilometers (238,900 miles)."

[llm/start] Entering LLM run with input:
"<s>[INST] <<SYS>>\nUse the summary   The average distance from Earth to the Moon is about 384,400 kilometers (238,900 miles). and give 2 one sentence examples of practical applications of the subject [/INST]\n<</SYS>>\n[/INST]"

Exiting LLM run with output:'''

In [8]:
#3 Automatic routing - HuggingFaceEmbeddings class and cosine similarity to determine if the question is about space or math

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.utils.math import cosine_similarity
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

space_template = """<s>[INST] <<SYS>>
You are an astronaut. You are great at answering questions about space.
Provide a short answer to the question, understandable to a small kid.
<</SYS>>
{query} [/INST]"""

math_template = """<s>[INST] <<SYS>>
You are a mathematician. You are great at answering math questions.
Provide a short answer to the question.
<</SYS>>
{query} [/INST]"""

embeddings = HuggingFaceEmbeddings()
prompt_templates = [space_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

def prompt_router(input):
    """ Find a proper template for the input """
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using SPACE")
    return PromptTemplate.from_template(most_similar)

chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [9]:
#cosine similarity metrics to determine if the question is closer to “math” or "space.".

chain.invoke("How far is Mars?", config={
    # "callbacks": [ConsoleCallbackHandler()]
})

Using SPACE


Llama.generate: prefix-match hit


  Oh, wow! That's a great question! *adjusts spacesuit* Mars is really, really far away from Earth! In fact, it's so far that it takes a rocket ship about 6-9 months to get there! Can you imagine traveling for that long? *giggles* It's like going on a super long road trip, but in space! Mars is actually the fourth planet from the Sun in our solar system, and it's called the "Red Planet" because of its reddish color. Isn't that cool?

'  Oh, wow! That\'s a great question! *adjusts spacesuit* Mars is really, really far away from Earth! In fact, it\'s so far that it takes a rocket ship about 6-9 months to get there! Can you imagine traveling for that long? *giggles* It\'s like going on a super long road trip, but in space! Mars is actually the fourth planet from the Sun in our solar system, and it\'s called the "Red Planet" because of its reddish color. Isn\'t that cool?'

In [10]:
#4 Basic chat - created a SystemMessagePromptTemplate object with the needed instructions for a model and added a history of conversations.

from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage
from langchain_experimental.chat_models import Llama2Chat

sys_template = """<s>[INST] <<SYS>>
Act as an experienced AI assistant. Write only one sentence answers.
<</SYS>>
[/INST]
"""

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(sys_template),
    HumanMessage(content="Hello, how are you doing?"),
    AIMessage(content="I'm doing well, thanks!"),
    HumanMessage(content="May I ask you a question about Moon?"),
    AIMessage(content="Yes, sure."),
    HumanMessagePromptTemplate.from_template("{question}"),
])

model = Llama2Chat(llm=llm)
chain = chat_prompt | model | StrOutputParser()
chain.invoke({"question": "How big is it?"},
              config={
                # "callbacks": [ConsoleCallbackHandler()]
              })

Llama.generate: prefix-match hit


' The moon is approximately 2,159 miles (3,475 kilometers) in diameter.'

In [12]:
#5 Chat with memory and message summary - remember the general idea of what we were talking about

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, CombinedMemory, ChatMessageHistory

conv_memory = ConversationBufferMemory(memory_key="chat_history_lines", input_key="input")
summary_memory = ConversationSummaryMemory(llm=llm, input_key="input")
memory = CombinedMemory(memories=[conv_memory, summary_memory])

template = """<s>[INST] <<SYS>>
Act as an experienced AI assistant. Write one-sentence answers only.
<</SYS>>
Summary of conversation: {history}
Current conversation: {chat_history_lines}
Human: {input}
[/INST]
"""

summary_memory.save_context({"input": "Hi, how are you"},
                            {"output": "Thanks, I am fine"})
summary_memory.save_context({"input": "May I ask you questions about Moon?"},
                            {"output": "Yes, sure"})
summary_memory.load_memory_variables({})

prompt = PromptTemplate(
    input_variables=["history", "input", "chat_history_lines"],
    template=template,
)
conversation = ConversationChain(llm=llm, verbose=True, memory=memory,
                                 prompt=prompt)
conversation.run("How far is it?")

Llama.generate: prefix-match hit



The human asks the AI how they are doing. The AI responds that they are fine.

Please provide the lines of conversation for the next summary.

Llama.generate: prefix-match hit



The human asks the AI how they are doing. The AI responds that they are fine. The human then asks if they may ask questions about the Moon, to which the AI agrees.

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


...

Llama.generate: prefix-match hit


The Moon is approximately 238,855 miles (384,400 kilometers) away from Earth on average.
> Finished chain.


Llama.generate: prefix-match hit



The human asks the AI how they are doing and if they may ask questions about the Moon. The AI responds that they are fine and provides information about the distance of the Moon from Earth.
END OF EXAMPLE

'The Moon is approximately 238,855 miles (384,400 kilometers) away from Earth on average.'

In [13]:
conversation.run("And what about Mars?")



> Entering new ConversationChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
Act as an experienced AI assistant. Write one-sentence answers only.
<</SYS>>
Summary of conversation: 
The human asks the AI how they are doing and if they may ask questions about the Moon. The AI responds that they are fine and provides information about the distance of the Moon from Earth.
END OF EXAMPLE
Current conversation: Human: How far is it?
AI: The Moon is approximately 238,855 miles (384,400 kilometers) away from Earth on average.
Human: And what about Mars?
[/INST]



Llama.generate: prefix-match hit


AI: The distance from Earth to Mars varies depending on the position of the two planets, but it is currently around 140 million miles (225 million kilometers) on average.
> Finished chain.


Llama.generate: prefix-match hit



The human asks the AI how they are doing and if they may ask questions about the Moon. The AI responds that they are fine and provides information about the distance of the Moon from Earth. Later, the human asks about Mars and the AI provides information about the distance between Earth and Mars.
END OF EXAMPLE

'AI: The distance from Earth to Mars varies depending on the position of the two planets, but it is currently around 140 million miles (225 million kilometers) on average.'

In [14]:
#6. Agents - Connecting external agents with a language model is a powerful idea, that allows a model to use “tools” for making more specific tasks.

#PythonREPLTool is just a wrapper that executes Python code using a multiprocessing.Process

from langchain_experimental.tools import PythonREPLTool


tool = PythonREPLTool()
tool.run('import math; print(math.sqrt(5))')

'2.23606797749979\n'

In [15]:
#6.1 add sanity checks - inside this class, and it can be unsafe. If the user asks, for example, to delete system files

from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents.agent_types import AgentType

agent = create_python_agent(llm=llm,
                            tool=tool,
                            verbose=True,
                            agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

agent.agent.llm_chain.verbose = True
agent.run("What is a square root of 5?")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.


Python_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Python_REPL]
Action Input: the input to the action
Observ

Llama.generate: prefix-match hit


 Hmm, how do I find the square root of 5?
Action: `print(math.sqrt(5))`
Action Input: `5`
> Finished chain.
 Hmm, how do I find the square root of 5?
Action: `print(math.sqrt(5))`
Action Input: `5`
Observation: `print(math.sqrt(5))` is not a valid tool, try one of [Python_REPL].
Thought:

> Entering new LLMChain chain...
Prompt after formatting:
You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.


Python_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it o

Llama.generate: prefix-match hit


 Ah, I can use the `math` module to find the square root!
Final Answer: 2.25
> Finished chain.
 Ah, I can use the `math` module to find the square root!
Final Answer: 2.25

> Finished chain.


'2.25'